In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
import pandas as pd
import numpy as np
import random
from sklearn.neighbors import NearestNeighbors
from sklearn.model_selection import train_test_split
import time

from sklearn.svm import SVC
from sklearn.model_selection import KFold
from sklearn.metrics import jaccard_score, f1_score, hamming_loss, accuracy_score
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV, StratifiedShuffleSplit

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB

## http://scikit.ml/api/skmultilearn.adapt.mlaram.html
!pip install scikit-multilearn
from skmultilearn.problem_transform import LabelPowerset

     |████████████████████████████████| 89 kB 3.6 MB/s 


In [4]:
DF = pd.read_csv("/content/gdrive/MyDrive/MI_Paper/4_Final_Dataset/PMM_train.csv")
TEST = pd.read_csv("/content/gdrive/MyDrive/MI_Paper/4_Final_Dataset/PMM_test.csv")

OUTCOME_old = ["FIBR_PREDS","PREDS_TAH","JELUD_TAH","FIBR_JELUD","A_V_BLOK","OTEK_LANC","RAZRIV","DRESSLER","ZSN","REC_IM","P_IM_STEN","LET_IS"]
OUTCOME = ["Arrhythmia","MyocardialDressler","CHF","MI/angina","LET_IS"]
FEATURES = list(set(list(DF.columns)).difference(set(OUTCOME)).difference(set(OUTCOME_old)))

DF_X = DF.loc[:,FEATURES]
DF_Y = DF.loc[:,OUTCOME]
TEST_X = TEST.loc[:,FEATURES]
TEST_Y = TEST.loc[:,OUTCOME]

In [5]:
def scatter_outcome(pre,new):
  fig, axes = plt.subplots(nrows=1, ncols=5, figsize=(24, 5))
  axe = axes.ravel()
  new['Arrhythmia'].value_counts().plot(ax = axe[0], kind='bar', title='Arrhythmia')
  pre['Arrhythmia'].value_counts().plot(ax = axe[0], marker='o', title='Arrhythmia',color='red')

  new['MyocardialDressler'].value_counts().plot(ax = axe[1], kind='bar', title='Myocardial or Dressler')
  pre['MyocardialDressler'].value_counts().plot(ax = axe[1], marker='o', title='Myocardial or Dressler',color='red')

  new['CHF'].value_counts().plot(ax = axe[2], kind='bar', title='Congestive Heart Failure')
  pre['CHF'].value_counts().plot(ax = axe[2], marker='o', title='Congestive Heart Failure',color='red')

  new['MI/angina'].value_counts().plot(ax = axe[3], kind='bar', title='Post-infarction MI/angina')
  pre['MI/angina'].value_counts().plot(ax = axe[3], marker='o', title='Post-infarction MI/angina',color='red')

  new['LET_IS'].value_counts().plot(ax = axe[4], kind='bar', title='Lethal Outcome')
  pre['LET_IS'].value_counts().plot(ax = axe[4], marker='o', title='Lethal Outcome',color='red')

In [6]:
ql=[0.05, 1.]
# Function to print the IRLbl values, measure for class imbalance
def print_irlbl (df):
    irlbl = df.sum(axis=0)
    #irlbl = irlbl[(irlbl < irlbl.quantile(ql[1]))]
    irlbl = irlbl.max() / irlbl
    print (irlbl)
    print("Mean: ", irlbl.mean())

# Function to find the underrepresented targets, which are defined as those observed less than the median occurance    
def get_tail_label(df: pd.DataFrame, ql=[0.05, 1.]) -> list:
    irlbl = df.sum(axis=0)
    #irlbl = irlbl[(irlbl > irlbl.quantile(ql[0])) & ((irlbl < irlbl.quantile(ql[1])))]  # Filtering
    #irlbl = irlbl[(irlbl < irlbl.quantile(ql[1]))]
    irlbl = irlbl.max() / irlbl
    threshold_irlbl = irlbl.median()
    tail_label = irlbl[irlbl > threshold_irlbl].index.tolist()
    return tail_label

# Function to get the minority samples from the dataset
# Function returns the feature vector minority dataframe as X_sub and target vector minority dataframe as y_sub
def get_minority_samples(X: pd.DataFrame, y: pd.DataFrame, ql=[0.05, 1.]):
    tail_labels = get_tail_label(y, ql=ql)
    index = y[y[tail_labels].apply(lambda x: (x == 1).any(), axis=1)].index.tolist()
    X_sub = X[X.index.isin(index)].reset_index(drop = True)
    y_sub = y[y.index.isin(index)].reset_index(drop = True)
    return X_sub, y_sub

# Function that gives the index of nearest neighbor of all the instances
def nearest_neighbour(X: pd.DataFrame, neigh) -> list:
    nbs = NearestNeighbors(n_neighbors=neigh, metric='euclidean', algorithm='kd_tree').fit(X)
    euclidean, indices = nbs.kneighbors(X)
    return indices

# Function that gets the augmented data using the MLSMOTE algorithm
# Input: X is the input vector DataFrame, y is the feature vector dataframe, n_sample is the number of newly generated sample
# Output: new_X is the augmented feature vector data, target is the augmented target vector data
def MLSMOTE(X, y, n_sample, neigh=3):
    indices2 = nearest_neighbour(X, neigh=5) # Get indices of the 5 nearest neighbors
    n = len(indices2)
    new_X = np.zeros((n_sample, X.shape[1]))
    target = np.zeros((n_sample, y.shape[1]))
    
    # Subsets of numerical and categorical columns for different methods of generating the x features
    Numerical=['AGE','S_AD_ORIT','D_AD_ORIT','K_BLOOD','NA_BLOOD','ALT_BLOOD','AST_BLOOD','L_BLOOD','ROE']
    No_Need=['ID']
    Categorical=list(set(list(X.columns)).difference(set(Numerical)).difference(set(No_Need)))
    
    for i in range(n_sample):
        reference = random.randint(0, n-1) # Randomly select 1 of the k nearest neighbors
        neighbor = random.choice(indices2[reference, 1:])
        all_point = indices2[reference]
        nn_df = y[y.index.isin(all_point)]
        ser = nn_df.sum(axis = 0, skipna = True)
        target[i] = np.array([1 if val > 0 else 0 for val in ser])  # Create synthetic y instance
        
        ratio = random.random()
        gap = X.loc[reference,Numerical] - X.loc[neighbor,Numerical]
        X_NUM = np.array(X.loc[reference,Numerical] + ratio * gap)
        X_CAT = np.array((X.loc[all_point,Categorical].mode()).loc[0])
        new_X[i] = np.concatenate((X_NUM, X_CAT), axis=0)
        
    new_X = pd.DataFrame(new_X, columns=Numerical+Categorical)
    target = pd.DataFrame(target, columns=y.columns)
    return new_X, target

def resample_dataset(train_X,train_Y):
  X_sub, y_sub = get_minority_samples(train_X, train_Y)  # Getting minority samples of that dataframe

  X_sub = train_X
  y_sub = train_Y

  #start_time = time.time()
  for i in range(12):
    X_min, y_min = get_minority_samples(X_sub, y_sub)  # Getting minority samples of that dataframe
    X_res, y_res = MLSMOTE(X_min, y_min, 40, 5)  # Applying MLSMOTE to augment the dataframe
    X_sub = pd.concat([X_sub,X_res])
    y_sub = pd.concat([y_sub,y_res])
  #print("--- %s seconds ---" % (time.time() - start_time))

  RES_X = X_sub
  RES_Y = y_sub
  
  return(X_sub,y_sub)

In [7]:
NUM_FOLD = 5
kf = KFold(n_splits=NUM_FOLD, random_state=1, shuffle=True) # Split the dataset into k folds
labels = DF_Y.columns[0:].tolist()

folds = list(kf.split(np.arange(len(DF))))
DF['fold'] = 0
for i in range(NUM_FOLD):
    DF['fold'][folds[i][1]] = i

# Check how well the folds are stratified.
print("fold                                         1    2    3    4    5   total")
print("==========================================================================")
for label in labels:
    label_padded = label + " "*(40-len(label))
    dist = ": "
    for i in range(NUM_FOLD):
        dist += "{:4d} ".format(DF[label][folds[i][1]].sum())
    dist += "{:4d} ".format(DF[label].sum())
    print(label_padded + dist)
label_padded = "total" + " "*(40-len("total"))
dist = ": "
for i in range(NUM_FOLD):
    dist += "{:4d} ".format(DF.iloc[folds[i][1]].shape[0])
dist += "{:4d} ".format(DF.shape[0])
print(label_padded + dist)

fold                                         1    2    3    4    5   total
Arrhythmia                              :   48   52   52   41   53  246 
MyocardialDressler                      :   17   22   19   19   27  104 
CHF                                     :   78   70   84   81   75  388 
MI/angina                               :   38   46   42   47   42  215 
LET_IS                                  :   47   40   31   39   53  210 
total                                   :  272  272  272  272  272 1360 


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [8]:
jac_train=[0]*NUM_FOLD
f1w_train=[0]*NUM_FOLD
f1m_train=[0]*NUM_FOLD
accu_train=[0]*NUM_FOLD
ham_train=[0]*NUM_FOLD

jac_test=[0]*NUM_FOLD
f1w_test=[0]*NUM_FOLD
f1m_test=[0]*NUM_FOLD
accu_test=[0]*NUM_FOLD
ham_test=[0]*NUM_FOLD

# Params for SVC
#par_C = [50, 80, 100, 200, 300] 
#par_gamma=[0.001, 0.01, 0.1]

# Params for logistic regression
par_pen = ['l1'] #'l1', 'l2'
par_C = [0.01] #0.001, 0.01, 0.1, 1, 10, 100

Numerical=['AGE','S_AD_ORIT','D_AD_ORIT','K_BLOOD','NA_BLOOD','ALT_BLOOD','AST_BLOOD','L_BLOOD','ROE']

for i in range(len(par_C)):
  for j in range(len(par_pen)):
    for fold, (train_index, test_index) in enumerate(kf.split(DF), 1):
      X_train = DF_X.iloc[train_index]
      Y_train = DF_Y.iloc[train_index]  
      X_test = DF_X.iloc[test_index]
      Y_test = DF_Y.iloc[test_index]

      X_train_SCALED = X_train.copy()
      X_testSCALED = X_test.copy()

      transformer = StandardScaler().fit(X_train_SCALED[Numerical])
      X_train_SCALED[Numerical] = transformer.transform(X_train_SCALED[Numerical])
      X_testSCALED[Numerical] = transformer.transform(X_testSCALED[Numerical])
    
      ######### Resample the dataset with MLSMOTE #########
      X_train_oversampled, Y_train_oversampled = resample_dataset(X_train_SCALED, Y_train)

      ######### Train the model (change the code here for another type of model)#########
      model = LabelPowerset(classifier = LogisticRegression(penalty = par_pen[j], C = par_C[i], solver = 'liblinear', max_iter=7000))
      #model = LabelPowerset(classifier = SVC(C=par_C[i], gamma=par_gamma[j], kernel='rbf'))
                        #GaussianNB()   
                        #LogisticRegression(penalty = 'l2', C = 100, solver = 'liblinear', max_iter=7000))
                               
      #model = LabelPowerset(classifier = GaussianNB())
      model.fit(X_train_oversampled, Y_train_oversampled)

      y_hat = model.predict(X_train_oversampled)
      Y_pred = model.predict(X_testSCALED)

      jac_train[fold-1] = jaccard_score(Y_train_oversampled, y_hat, average='weighted')
      f1w_train[fold-1] = f1_score(Y_train_oversampled, y_hat, average='weighted')
      f1m_train[fold-1] = f1_score(Y_train_oversampled, y_hat, average='micro')
      accu_train[fold-1] = accuracy_score(Y_train_oversampled, y_hat)
      ham_train[fold-1] = hamming_loss(Y_train_oversampled, y_hat) 
    
      jac_test[fold-1] = jaccard_score(Y_test, Y_pred, average='weighted')
      f1w_test[fold-1] = f1_score(Y_test, Y_pred, average='weighted')
      f1m_test[fold-1] = f1_score(Y_test, Y_pred, average='micro')
      accu_test[fold-1] = accuracy_score(Y_test, Y_pred)
      ham_test[fold-1] = hamming_loss(Y_test, Y_pred)


    ######### Print the overall performance accross all folds (test set) #########
    print("============================================================================")
    print(f'For C={par_C[i]} and pen={par_pen[j]}:')
    print("          F1-w ", "F1-mi ", "Jaccard ", "Accuracy ", "Hamming")
    # Training set: print result for overleaf
    print("Training: ","& ",round(sum(f1w_train)/len(f1w_train),3),"& ", round(sum(f1m_train)/len(f1m_train),3), 
      "& ",round(sum(jac_train)/len(jac_train), 3),"& ",round(sum(accu_train)/len(accu_train), 3),
      "& ",round(sum(ham_train)/len(ham_train),3))
    
    # Validation set: print result for overleaf
    print("Validation: ","& ",round(sum(f1w_test)/len(f1w_test),3),"& ", round(sum(f1m_test)/len(f1m_test),3), 
      "& ",round(sum(jac_test)/len(jac_test), 3),"& ",round(sum(accu_test)/len(accu_test), 3),
      "& ",round(sum(ham_test)/len(ham_test),3))


/usr/local/lib/python3.7/dist-packages/sklearn/base.py:444: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:444: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:444: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:444: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:444: UserWarning: X has feature names, but Lo

For C=0.01 and pen=l1:
          F1-w  F1-mi  Jaccard  Accuracy  Hamming
Training:  &  0.467 &  0.467 &  0.306 &  0.324 &  0.37
Validation:  &  0.29 &  0.28 &  0.172 &  0.332 &  0.305


In [9]:
Numerical=['AGE','S_AD_ORIT','D_AD_ORIT','K_BLOOD','NA_BLOOD','ALT_BLOOD','AST_BLOOD','L_BLOOD','ROE']
DF_X_SCALED = DF_X.copy()
TEST_XSCALED = TEST_X.copy()
transformer = StandardScaler().fit(DF_X_SCALED[Numerical])
DF_X_SCALED[Numerical] = transformer.transform(DF_X_SCALED[Numerical])
TEST_XSCALED[Numerical] = transformer.transform(TEST_XSCALED[Numerical])

######### Train the model (change the code here for another type of model)#########
#model = LabelPowerset(classifier = GaussianNB())
#model = LabelPowerset(classifier = SVC(C=200, gamma=0.001,kernel='rbf')) # SVC(C=0.1, kernel='rbf', gamma=1)
model = LabelPowerset(classifier = LogisticRegression(penalty = 'l1', C = 100, solver = 'liblinear', max_iter=7000))
model.fit(DF_X_SCALED, DF_Y)
test_pred = model.predict(TEST_XSCALED)

# Testing set: print result for overleaf
print("============================================================================")
print("          F1-w ", "F1-mi ", "Jaccard ", "Accuracy ", "Hamming")
print("Testing: ","& ",round(f1_score(TEST_Y, test_pred, average='weighted'), 3),"& ", round(f1_score(TEST_Y, test_pred, average='micro'),3), 
  "& ",round(jaccard_score(TEST_Y, test_pred, average='weighted'), 3),"& ",round(accuracy_score(TEST_Y, test_pred), 3),
  "& ",round(hamming_loss(TEST_Y, test_pred),3))

          F1-w  F1-mi  Jaccard  Accuracy  Hamming
Testing:  &  0.322 &  0.327 &  0.201 &  0.341 &  0.232


/usr/local/lib/python3.7/dist-packages/sklearn/base.py:444: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"
